In [80]:
import pyvisa
import time

rm = pyvisa.ResourceManager()
print(rm.list_resources())

('USB0::0xF4ED::0xEE3A::SDG08CBX5R0530::INSTR', 'USB0::0xF4ED::0xEE3A::SDG08CBX7R0701::INSTR', 'ASRL3::INSTR', 'ASRL4::INSTR')


In [82]:
rm = pyvisa.ResourceManager()
inst1 = rm.open_resource(rm.list_resources()[0])  # Use your exact address
inst2 = rm.open_resource(rm.list_resources()[1])
print(inst1.query("*IDN?"))
# Set frequency and amplitude if needed

inst1.write('OUTP1 ON')
# Slave: enable gated burst (external trigger)
inst2.write('C1:BSWV GATE,EXT')        # switch to external gating
inst2.write('C1:BSWV CYCLES,INF')      # infinite cycles
inst2.write('OUTP1 ON')

inst1.write('C1:BSWV FRQ,100;C1:BSWV PHSE,00;C1:BSWV AMP,3')#inst1.write('C1:BSWV AMP,2')          # 2Vpp
inst2.write('C1:BSWV FRQ,100.0;C1:BSWV PHSE,90;C1:BSWV AMP,3')        # 1kHz


# Change phase dynamically
#for phase in range(0, 360, 10):  # from 0 to 350 degrees
#    inst1.write(f'C1:BSWV PHSE,{phase}')
#    time.sleep(1)  # delay between updates

inst1.close()
inst2.close()
rm.close()

*IDN SDG,SDG830,SDG08CBX5R0530,1.08.01.15,06-00-00-12-00



In [48]:
boven = 'USB0::0xF4ED::0xEE3A::SDG08CBX7R0713::INSTR'

In [75]:
rm = pyvisa.ResourceManager()
inst1 = rm.open_resource(rm.list_resources()[0])  # Trigger master
inst2 = rm.open_resource(rm.list_resources()[1])  # Trigger slave

# Optional: Identify devices
print("Master:", inst1.query("*IDN?"))
print("Slave:", inst2.query("*IDN?"))

# --- Configure Channel 1 (Sine) ---
inst1.write("C1:BSWV WVTP,SINE")        # Sine wave
inst1.write("C1:BSWV FRQ,1000")         # 1kHz
inst1.write("C1:BSWV AMP,3")            # 3Vpp
inst1.write("C1:BSWV PHSE,0")           # 0° phase
inst1.write("C1:BTWV STATE,ON")         # Enable burst mode
inst1.write("C1:BTWV MODE,TRIG")        # Triggered burst
inst1.write("C1:BTWV TRSR,MAN")         # Manual trigger
inst1.write("C1:BTWV CYC,INF")          # Infinite cycles after trigger
inst1.write("C1:OUTP ON")               # Arm output (await trigger)

# --- Configure Channel 2 (Cosine, 90° phase shift) ---
inst2.write("C1:BSWV WVTP,SINE")
inst2.write("C1:BSWV FRQ,1000")
inst2.write("C1:BSWV AMP,3")
inst2.write("C1:BSWV PHSE,90")          # 90° = cosine
inst2.write("C1:BTWV STATE,ON")
inst2.write("C1:BTWV MODE,TRIG")
inst2.write("C1:BTWV TRSR,EXT")         # Wait for external trigger (from Inst1)
inst2.write("C1:BTWV CYC,INF")
inst2.write("C1:OUTP ON")               # Arm output

time.sleep(0.5)  # Wait for settings to apply

inst1.write("C1:TRIG")                   # Fire manual trigger (Inst2 receives pulse via BNC)

# Done: Both channels now running, 90° apart

Master: *IDN SDG,SDG830,SDG08CBX7R0713,1.08.01.15R3,08-00-00-13-00

Slave: *IDN SDG,SDG830,SDG08CBX5R0530,1.08.01.15,06-00-00-12-00



9